In [1]:
import numpy as np
from pycomposer.composition_controler import CompositionControler
from pycomposer.inferablepitch.rtrbm_inferer import RTRBMInferer
from pycomposer.inferableconsonance.q_consonance import QConsonance
from pycomposer.midi_vectorlizer import MidiVectorlizer

In [2]:
midi_vectorlizer = MidiVectorlizer()
tone_df = midi_vectorlizer.extract("../../furudokei.mid")
tone_df = tone_df.sort_values(by=["start", "end"])

In [3]:
inferable_consonance = QConsonance()
inferable_consonance.initialize()
for pitch in tone_df.pitch.values.tolist():
    inferable_consonance.learn(state_key=pitch, limit=2)

In [4]:
inferable_pitch = RTRBMInferer(
    learning_rate=0.00001,
    hidden_n=100,
    hidden_binary_flag=True,
    inferancing_training_count=1,
    r_batch_size=200
)

In [5]:
inferable_pitch.learn(tone_df=tone_df, training_count=1, batch_size=200)

In [13]:
composition_controler = CompositionControler(
    resolution=960,
    initial_tempo=120
)

In [14]:
chord_list = composition_controler.create_chord_list(
    octave=4, 
    first_chord="I",
    measure_n=20
)

composition_controler.compose_chord(
    chord_list,
    metronome_time=100, 
    start_measure_n=0,
    chord_instrument_num=33,
    chord_velocity_range=(70, 90)
)

In [15]:
composition_controler.compose_melody(
    inferable_pitch,
    inferable_consonance,
    chord_list,
    measure_n=20,
    start_measure_n=0,
    beat_n=4,
    metronome_time=100,
    melody_instrument_num=0,
    melody_velocity_range=(90, 110)
)

In [16]:
composition_controler.save("test3.mid")